# CloudML with DataLAB Part. 4

### How to deploy the Prediction Model

Hello! I'm back! :)

This time, we're going to learn how to deploy your trained model

Tensorflow provides the function that you can serve the model.

As you know it's tooooooooooo difficault

But!!! If you deploy using CloudML, That comes to be so easy

Let's see how to do

I'm going to build the simple Neural Network Model.

This model would take 10dims input vector like [0, 1, 2, .... 9] and [9, 8, 7, .. , 0] so predicts whether increase or decrease

In [81]:
import tensorflow as tf

data = [[i for i in range(10)], [9-i for i in range(10)]]
x = tf.placeholder(tf.float32, [None, 10])
feed_dict = {x: data}

layer1 = tf.layers.dense(x, 10)
layer2 = tf.layers.dense(layer1, 10)
layer3 = tf.layers.dense(layer2, 10)
logits = tf.layers.dense(layer3, 1)

with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  out = sess.run(tf.nn.sigmoid(logits), feed_dict=feed_dict)
  print(out)

[[ 0.80873543]
 [ 0.999708  ]]


Of course it's not a trained

I will take the neural network model from this code, and train on CloudML at first

Let's make a data file to simulate 

In [2]:
import csv
with open('DataSet/train_data.csv', 'w') as f:
  writer = csv.writer(f)
  for i in range(1000):
    if i % 2 == 0:
      writer.writerow([1]+[i for i in range(10)])
    else:
      writer.writerow([0]+[9-i for i in range(10)])
print("data has written")

data has written


Let's checke whether it saved well by reading it

All the element in column 0 means label.

you can see the 1 label means a increase data and 0 means opposit

In [5]:
with open('DataSet/train_data.csv', 'r') as f:
  reader = csv.reader(f)
  data = list(reader)
  for i in range(10):
    print(data[i])

['1', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
['0', '9', '8', '7', '6', '5', '4', '3', '2', '1', '0']
['1', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
['0', '9', '8', '7', '6', '5', '4', '3', '2', '1', '0']
['1', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
['0', '9', '8', '7', '6', '5', '4', '3', '2', '1', '0']
['1', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
['0', '9', '8', '7', '6', '5', '4', '3', '2', '1', '0']
['1', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
['0', '9', '8', '7', '6', '5', '4', '3', '2', '1', '0']


Now, I'll copy it to my bucket

In [40]:
%bash
PROJECT_ID=tensorflowprojects
BUCKET="gs://${PROJECT_ID}-mlengine"

gsutil cp -r ./DataSet ${BUCKET}/DataSet

Copying file://./DataSet/train_data.csv [Content-Type=text/csv]...
/ [1 files][ 22.5 KiB/ 22.5 KiB]                                                
Operation completed over 1 objects/22.5 KiB.                                     


Now, we're going to see the huge code. 

Don't freak out! You can use this code by copy and change a bit.

Our purpose is setting the Experiment function to run learn_runner.

Here's the details about this experiment function (document in tensorflow.org)
https://www.tensorflow.org/api_docs/python/tf/contrib/learn/Experiment

And the expriment function takes input arguments such Input function, Estimator, Metrics, Serving function which are defined below

I'm going to write this code in simple_model.py file using the magic %%writefile line

You can read the commants to refer the detal of each functions if you want

In [10]:
%%writefile ./model/simple_model.py
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf
from tensorflow.contrib import learn
from tensorflow.contrib import layers
from tensorflow.contrib import metrics
from tensorflow.contrib.learn.python.learn.estimators import model_fn as model_fn_lib

tf.logging.set_verbosity(tf.logging.INFO)

#
# 1. Input function
#
def input_fn(filename, batch_size=20):
  """
  We use this function to read the train data and build the batch
  If you have used to use the queue runners, go ahead at same
  Just care for the returns.
  you can see the return dictionary {'data': b_train}
  this form is for feed data from external query
  When you call the model to predict, you would use key for input name, value for input data to predict. That's why
  """
  filename_queue = tf.train.string_input_producer([filename])
  reader = tf.TextLineReader()
  _, value = reader.read(filename_queue)
  dataset = tf.decode_csv(value, [[0]]*11)
  label = tf.cast(dataset[0], tf.float32)
  train = tf.cast(tf.stack(dataset[1:]), tf.float32)

  b_train, b_label = tf.train.batch([train, label], batch_size)
  return {'data':b_train}, tf.reshape(b_label, (-1, 1))


def get_input_fn(filename, batch_size):
  """
  and you see the this function returns the input_fn as function type. 
  it's not a big deal. it's required by experiment function
  """
  return lambda: input_fn(filename, batch_size)


#
# 2. Estimator
#
def _simple_model_fn(features, labels, mode):
  """
  This function is just a model that you built
  you can see the input argument 'features'
  Actually it's a placeholders wrapped by dictionary 
  It's considered that any user who call this model inputs a data as a dict type to predict
  """
  x = features['data']
  layer1 = tf.layers.dense(x, 10)
  layer2 = tf.layers.dense(layer1, 10)
  layer3 = tf.layers.dense(layer2, 10)
  logits = tf.layers.dense(layer3, 1)

  loss = None
  train_op = None

  
  """
  Mode key means whether this model is training or not
  You can make a train_op and calculate loss like below
  """
  # Calculate Loss (for both TRAIN and EVAL modes)
  if mode != learn.ModeKeys.INFER:
    loss = tf.losses.sigmoid_cross_entropy(labels, logits)

  # Configure the Training Op (for TRAIN mode)
  if mode == learn.ModeKeys.TRAIN:
    train_op = tf.contrib.layers.optimize_loss(
      loss=loss,
      global_step=tf.contrib.framework.get_global_step(),
      learning_rate=0.001, optimizer="Adam")

  """
  Then you can make the prediction form and return ModelFnOp object like below
  """
  # Generate Predictions
  predictions = {
      "classes": tf.argmax(input=logits, axis=1),
      "probabilities": tf.nn.sigmoid(logits)
  }

  # Return a ModelFnOps object
  return model_fn_lib.ModelFnOps(mode=mode, loss=loss, train_op=train_op,
                               predictions=predictions)


def build_estimator(model_dir):
  """
  Lastly, build the estimator. Yeah just copy and paste. who knows?
  """
  return learn.Estimator(
         model_fn=_simple_model_fn,
         model_dir=model_dir,
         config=tf.contrib.learn.RunConfig(save_checkpoints_secs=180))


#
# 3. Evaluate metrics
#
def get_eval_metrics():
  """
  This function returns which metrics would like you see for evaluating
  These links would be help
  https://www.tensorflow.org/api_docs/python/tf/contrib/learn/MetricSpec
  https://www.tensorflow.org/api_docs/python/tf/metrics/accuracy
  """
  return {"accuracy": learn.MetricSpec(metric_fn=tf.metrics.accuracy,
                                     prediction_key="classes")
         }

#
# 4. Serving function
#
def serving_input_fn():
  """
  You saw the placeholder dictionary like {'data': tensor } form. 
  'data' is the name of input. I will be query to model with input data like {'data': [0, 1, 2, 3..., 9]}
  You can make the number of inputs in dictionary here
  Make the feature_placeholders in dictionary like below, and copy and paste otherwise. who knows!!
  """
  feature_placeholders = {'data': tf.placeholder(tf.float32, [None, 10])}
  features = {
    key: tensor
    for key, tensor in feature_placeholders.items()
  }    
  return learn.utils.input_fn_utils.InputFnOps(
    features,
    None,
    feature_placeholders
  )


Overwriting ./model/simple_model.py


At last! All functions are defined for experiment function

Here's the full code to build the experiment object 

You might need to change them hardly 

Just make experiment object following this code

In [11]:
%%writefile ./model/task.py

import argparse
import json
import os

import simple_model as model

import tensorflow as tf
from tensorflow.contrib.learn import Experiment
from tensorflow.contrib.learn.python.learn import learn_runner
from tensorflow.contrib.learn.python.learn.utils import (
    saved_model_export_utils)

def generate_experiment_fn(data_dir=None,
                           train_batch_size=100,
                           eval_batch_size=100,
                           train_steps=10000,
                           eval_steps=100,
                           **experiment_args):
  
  def _experiment_fn(output_dir):
    return Experiment(
      model.build_estimator(output_dir),
      train_input_fn=model.get_input_fn(
        filename=os.path.join(data_dir, "train_data.csv"),
        batch_size=train_batch_size),
      eval_input_fn=model.get_input_fn(
        filename=os.path.join(data_dir, "train_data.csv"),
        batch_size=eval_batch_size),

      export_strategies=[saved_model_export_utils.make_export_strategy(
          model.serving_input_fn,
          default_output_alternative_key=None,
          exports_to_keep=1
        )],
      train_steps=train_steps,
      eval_metrics=model.get_eval_metrics(),
      eval_steps=eval_steps,
      **experiment_args
      )
  return _experiment_fn


if __name__ == '__main__':
  parser = argparse.ArgumentParser()
  parser.add_argument(
      '--data_dir',
      help='GCS or local path to training data',
      required=True
  )
  parser.add_argument(
      '--train_batch_size',
      help='Batch size for training steps',
      type=int,
      default=100
  )
  parser.add_argument(
      '--eval_batch_size',
      help='Batch size for evaluation steps',
      type=int,
      default=100
  )
  parser.add_argument(
      '--train_steps',
      help='Steps to run the training job for.',
      type=int,
      default=10000
  )
  parser.add_argument(
      '--eval_steps',
      help='Number of steps to run evalution for at each checkpoint',
      default=100,
      type=int
  )
  parser.add_argument(
      '--output_dir',
      help='GCS location to write checkpoints and export models',
      required=True
  )
  parser.add_argument(
      '--job-dir',
      help='this model ignores this field, but it is required by gcloud',
      default='junk'
  )
  parser.add_argument(
      '--eval_delay_secs',
      help='How long to wait before running first evaluation',
      default=10,
      type=int
  )
  parser.add_argument(
      '--min_eval_frequency',
      help='Minimum number of training steps between evaluations',
      default=1,
      type=int
  )

  args = parser.parse_args()
  arguments = args.__dict__

  # unused args provided by service
  arguments.pop('job_dir', None)
  arguments.pop('job-dir', None)

  output_dir = arguments.pop('output_dir')

  # Run the training job
  learn_runner.run(generate_experiment_fn(**arguments), output_dir)

Overwriting ./model/task.py


Let's see we've done right!

You can confirm whether the model has an error using CloudML in local

Follow this shell commands

Note: -- \ argument splites the input arguments in two.

Left sides are for ml-engint, othersides are for model

In [12]:
%bash
PROJECT_ID=tensorflowprojects
BUCKET="gs://${PROJECT_ID}-mlengine"
JOB_NAME=simple_code_in_cloudml_test_$(date +%Y%m%d_%H%M%S)
PACKAGE_PATH=$(pwd)/model
MODULE_NAME=model.task
STAGING_BUCKET=${BUCKET}
JOB_DIR=${BUCKET}/${JOB_NAME}
OUTPUT=${BUCKET}/${JOB_NAME}
REGION=europe-west1
SCALE_TIER=BASIC_GPU

# Submit job with these settings
gcloud ml-engine local train \
--package-path=$PACKAGE_PATH \
--module-name=$MODULE_NAME \
-- \
--data_dir=${BUCKET}/DataSet \
--output_dir=${BUCKET}/${JOB_NAME} \
--train_step=3 \
--eval_step=3

INFO:tensorflow:Using config: {'_save_checkpoints_secs': 180, '_num_ps_replicas': 0, '_keep_checkpoint_max': 5, '_tf_random_seed': None, '_task_type': None, '_environment': u'cloud', '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f334ddebad0>, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_task_id': 0, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_evaluation_master': '', '_keep_checkpoint_every_n_hours': 10000, '_master': ''}
Instructions for updating:
Monitors are deprecated. Please use tf.train.SessionRunHook.
INFO:tensorflow:Create CheckpointSaverHook.
W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use SSE3 instructions, but these are available on your machine and could speed up CPU computations.
W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use SSE4.1 instructions, but these are available on your

There we go! 

Now it can be submitted by JOB on CloudML

Submit your JOB following commands in JOB and waiting for it training done

In [1]:
%bash
PROJECT_ID=tensorflowprojects
BUCKET="gs://${PROJECT_ID}-mlengine"
JOB_NAME=simple_model_$(date +%Y%m%d_%H%M%S)
PACKAGE_PATH=$(pwd)/model
MODULE_NAME=model.task
STAGING_BUCKET=${BUCKET}
JOB_DIR=${BUCKET}/${JOB_NAME}
OUTPUT=${BUCKET}/${JOB_NAME}
REGION=europe-west1
SCALE_TIER=BASIC_GPU

# Submit job with these settings
gcloud ml-engine jobs submit training $JOB_NAME \
--package-path=$PACKAGE_PATH \
--module-name=$MODULE_NAME \
--staging-bucket=$STAGING_BUCKET \
--job-dir=$JOB_DIR \
--scale-tier=$SCALE_TIER \
--region=$REGION \
-- \
--data_dir=${BUCKET}/DataSet \
--output_dir=${BUCKET}/${JOB_NAME} \
--train_step=100 \
--eval_step=100

jobId: simple_model_20170614_230005
state: QUEUED


Job [simple_model_20170614_230005] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ml-engine jobs describe simple_model_20170614_230005

or continue streaming the logs with the command

  $ gcloud ml-engine jobs stream-logs simple_model_20170614_230005


Well, mine is Done!

Now, only deploy the model is remained

You can find your model following command

In [15]:
%bash
echo $(gsutil ls gs://tensorflowprojects-mlengine/simple_model_20170611_050925/export/Servo | tail -1)

gs://tensorflowprojects-mlengine/simple_model_20170611_050925/export/Servo/1497157932533/


Then, Deploy! two lines are enough

In [16]:
%bash
REGION='us-central1'
MODEL_NAME="simple_model"
MODEL_VERSION="v1"
MODEL_LOCATION=$(gsutil ls gs://tensorflowprojects-mlengine/simple_model_20170611_050925/export/Servo | tail -1)

gcloud ml-engine models create ${MODEL_NAME} --regions $REGION
gcloud ml-engine versions create ${MODEL_VERSION} --model ${MODEL_NAME} --origin ${MODEL_LOCATION}

Creating version (this might take a few minutes)......
..........................................................................done.


That's all you have done!

Let's check it out!

The prediction model you've deployed can be called by query with json file which input data is written down

I don't have any input data so I made it

In [23]:
import json

with open("request.json", "w") as f:
  for i in range(10):
    if i < 5:
      f.write(json.dumps({"inputs": [9-i for i in range(10)]}) + '\n')
    else:
      f.write(json.dumps({"inputs": [i for i in range(10)]}) + '\n')
      
print("json file written")

json file written


Then, request prediction model using this file

In [24]:
%bash
MODEL_NAME="simple_model"
gcloud ml-engine predict --model ${MODEL_NAME} --json-instances request.json

CLASSES  PROBABILITIES
0        [1.2494094789872179e-06]
0        [1.2494094789872179e-06]
0        [1.2494094789872179e-06]
0        [1.2494094789872179e-06]
0        [1.2494094789872179e-06]
0        [0.9999992847442627]
0        [0.9999992847442627]
0        [0.9999992847442627]
0        [0.9999992847442627]
0        [0.9999992847442627]


You got this!!! You all have done!!

CLASSES is for multi-class classfication

But this model predicts by binary so 0 value of CLASSES is right to be shown

That's all. I think this part was pretty hard to follow than previous

Here's some references to help following this part easy

Now, enjoy!

### Reference:

CloudML Prediction API
https://github.com/GoogleCloudPlatform/cloudml-dist-mnist-example

deploy document https://cloud.google.com/ml-engine/docs/concepts/prediction-overview

exmaple https://github.com/GoogleCloudPlatform/training-data-analyst/blob/master/blogs/babyweight/babyweight.ipynb